In [28]:
import pandas as pd
df = pd.read_csv('final_eto_output_azure.csv')
df.dropna(subset=df.columns, inplace=True)


In [29]:
df.columns = df.iloc[0]
df = df[1:]
df.rename(columns={'1992 ETo April.pdf': 'source'}, inplace=True)
df.dropna(how='all',inplace=True)

In [30]:
# all are nummeric columns except 'source' column, so remove all non-numeric values from numeric columns
for col in df.columns:
    if col != 'source':
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [31]:
df['year'] = df['source'].str.extract(r'(\d{4})').astype(int)
df['year'].unique()

array([1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024,
       1990, 1991])

In [32]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
df['month'] = df['source'].str.extract(r'(' + '|'.join(months) + r')')
#replace nan to February
df['month'].fillna('February', inplace=True)
df['month'] = pd.to_datetime(df['month'], format='%B').dt.month
df['month'].unique()

C:\Users\infyz\AppData\Local\Temp\ipykernel_10300\839321967.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['month'].fillna('February', inplace=True)


array([ 4,  2, 12,  3,  5, 11,  1,  7,  6,  8,  9, 10])

In [33]:
df['Day'].dropna(inplace=True)
df.dropna(subset=['Day'], inplace=True)
df['Day'] = df['Day'].astype(int)

In [35]:
#got this error "ValueError: day is out of range for month" because some days are invalid for some months like 30th February etc., handle that by dropping those rows
df = df[pd.to_datetime(dict(year=df['year'], month=df['month'], day=df['Day']), errors='coerce').notnull()]

In [37]:
df['date'] = pd.to_datetime(df[['year', 'month', 'Day']])

C:\Users\infyz\AppData\Local\Temp\ipykernel_10300\1568190000.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df[['year', 'month', 'Day']])


In [39]:
df.drop(columns=['source', 'year', 'month', 'Day'], inplace=True)

C:\Users\infyz\AppData\Local\Temp\ipykernel_10300\748070575.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['source', 'year', 'month', 'Day'], inplace=True)


In [41]:
df.to_csv('final_eto_output_azure_cleaned.csv', index=False)